In [2]:
"""%pip install tensorflow
%pip install tensorflow-datasets
%pip install transformers
%pip install torch
%pip install datasets
%pip install scikit-learn
%pip install pandas
%pip install numpy
%pip install transformers
%pip install sklearn"""

'%pip install tensorflow\n%pip install tensorflow-datasets\n%pip install transformers\n%pip install torch\n%pip install datasets\n%pip install scikit-learn\n%pip install pandas\n%pip install numpy\n%pip install transformers\n%pip install sklearn'

In [35]:
#Importing numerous libraries
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D
import torch
import torch.utils.data as tud
import numpy as np
from transformers import AutoModelForSequenceClassification
import pandas as pd
from datasets import Dataset, load_dataset
import re
import json
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.utils import resample
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from typing_extensions import Self
from google.colab import drive







In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [5]:
# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("boltuix/emotions-dataset")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

emotions_dataset.parquet:   0%|          | 0.00/7.41M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/131306 [00:00<?, ? examples/s]

In [6]:
ds['train']

Dataset({
    features: ['Sentence', 'Label'],
    num_rows: 131306
})

In [7]:
data = pd.DataFrame(data=ds['train'], columns=["Sentence", "Label"])

In [8]:
X, y = data["Sentence"], data["Label"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [10]:
X_train, y_train = pd.DataFrame(data=X_train, columns=["Sentence"]), pd.DataFrame(data=y_train, columns=["Label"])
X_test, y_test = pd.DataFrame(data=X_test, columns=["Sentence"]), pd.DataFrame(data=y_test, columns=["Label"])


In [11]:
X_train

,Sentence
128324,I see BOTW and now Im waiting for a group of g...
14873,Theres a BIG difference between being against ...
82295,i have tried sorting out the area for the cat ...
67540,I m that skinny and have so much trouble gaini...
103337,i am grateful to have a strong support system ...
...,...
54886,r vaxxhappened Fuck you Jan
110268,Thats a great dream. I love how your inner voi...
119879,Oh my god Don t you know it s illegal to share...
103694,"I’m not gonna lie guys, if we help a team brea..."


In [12]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
print("Tokenizer vocab size:", tokenizer.vocab_size)  # Might be > 30153

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Tokenizer vocab size: 28996


In [13]:
encoded = tokenizer(
    X_train['Sentence'].tolist(),
    padding="max_length",
    truncation=True,
    max_length=64,
    return_tensors="pt"
)

X_train['input_ids'] = encoded['input_ids'].tolist()
X_train['attention_mask'] = encoded['attention_mask'].tolist()

In [14]:
encoded = tokenizer(
    X_test['Sentence'].tolist(),
    padding="max_length",
    truncation=True,
    max_length=64,
    return_tensors="pt"
)

X_test['input_ids'] = encoded['input_ids'].tolist()
X_test['attention_mask'] = encoded['attention_mask'].tolist()

In [15]:
def show_labelAmts(data):
    for i in data['label'].unique():
        print(len(data[data['label'] == i]), i)

In [16]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()

    return str(text)

In [17]:
labelsMap = {}
val = 0
for labels in y.unique():
    labelsMap[labels] = val
    val += 1

In [18]:
labelsMap

{'happiness': 0,
 'neutral': 1,
 'sadness': 2,
 'surprise': 3,
 'love': 4,
 'fear': 5,
 'confusion': 6,
 'disgust': 7,
 'desire': 8,
 'shame': 9,
 'sarcasm': 10,
 'anger': 11,
 'guilt': 12}

In [19]:
def encodeLabels(data, mapping):
    for item in mapping:
        data.replace({'Label', item}, mapping.get(item), inplace=True)

In [20]:
encodeLabels(y_train, labelsMap)
encodeLabels(y_test, labelsMap)

/tmp/ipython-input-493815607.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'Label', item}, mapping.get(item), inplace=True)


In [21]:
class EmotionTextDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        df = pd.DataFrame(data=X, columns=["Sentence", "input_ids", "attention_mask"])
        df["Label"] = y

        df = df.reset_index(drop=True)

        self.input_ids = df['input_ids']
        self.attention_mask = df['attention_mask']
        self.labels = df["Label"]
        self.classes = len(df["Label"].unique())

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float32)  # or long if single-label
        }

    def __len__(self):
        return len(self.labels)

In [22]:
from collections import Counter

token_counts = Counter([token for tokens in X_train['input_ids'] for token in tokens])
vocab_size = max(token_counts) + 1  # Assuming tokens start at 0
vocab_size

28732

In [23]:
dataSetTrain = EmotionTextDataset(X_train, y_train)
dataloaderTrain = tud.DataLoader(dataSetTrain, batch_size=64, shuffle=True)



In [24]:
class StatReporter:
    def start(self):
        self.start_time = time.time()
        self.elapsed = 0
        self.target = 0
        self.loss = None

    def iteration(self, epoch, i, loss):
        self.loss = loss

        iteration_time = time.time() - self.start_time
        self.elapsed += iteration_time
        self.start_time = time.time()
        if self.elapsed > self.target:
            print(f"Epoch {epoch+1:2d}, iteration {i+1:3d}:  Loss = {loss:.3f}  Iteration time = {iteration_time:0.3f}")
            self.target += 10

    def end(self):
        print(f"Training complete.  Elapsed time: {self.elapsed:.2f} seconds  Final loss: {self.loss:0.3f}")

stats = StatReporter()  # one StatReporter object to use throughout

In [25]:
#custom model
class EmotionBiLSTM(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_labels):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.lstm = torch.nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2)
        self.dropout = torch.nn.Dropout(0.5)
        self.norm = torch.nn.LayerNorm(hidden_dim * 2)
        self.fc1 = torch.nn.Linear(hidden_dim * 2, 64)
        self.fc2 = torch.nn.Linear(64, num_labels)

        self.act = torch.nn.GELU()



    def forward(self, input_ids, attention_mask=None):
        x = self.embedding(input_ids)

        if attention_mask is not None:
            lengths = attention_mask.sum(dim=1).cpu()
            x = torch.nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)

        _, (hn, _) = self.lstm(x)

        # Concatenate final hidden states from both directions
        x = torch.cat((hn[-2], hn[-1]), dim=1)

        x = self.norm(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = torch.nn.functional.log_softmax(x, dim=1)


        return x




In [26]:
from typing_extensions import Self
def train_model(model, dataloader, epochs):
    stats.start()
    model.to(device)
    #weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, eps=1e-06, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',         # Minimize loss
    factor=0.5,         # Reduce LR by half
    patience=3         # Wait 3 epochs before reducing
)
    i = 0

    for epoch in range(epochs):
        running_loss = 0.0

        for data in dataloader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['labels'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            optimizer.zero_grad()
            #Forward Propagation

            #back propagation
            loss = criterion(outputs, labels.long())
            loss.backward()

            #updating the weights
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            i += 1

        #keeping track after each iteration of training
        stats.iteration(epoch, i, loss)

        #Early stopping
        if loss < .400:
            stats.end()
            break
        scheduler.step(loss.item())


    stats.end()

In [27]:
#Test model function provided by Mark Liffiton
def test_model(model, dataloader):
    classes = dataloader.dataset.classes
    class_correct = {clsname: 0 for clsname in classes}
    class_total = {clsname: 0 for clsname in classes}
    total_correct = 0
    total = 0

    with torch.no_grad():
        for data in dataloader:


            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['labels'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)



            # get predictions from multiple class outputs
            _, predicted = torch.max(outputs, 1)
            # find and count the correct predictions
            corrects = (predicted == labels).squeeze()
            total_correct += corrects.sum()
            total += outputs.shape[0]

            # count correct predictions within each clsas
            for label, correct in zip(labels, corrects):
                clsname = classes[int(label)]
                class_correct[clsname] += correct.item()
                class_total[clsname] += 1

    # compute overall accuracies
    accuracy = (total_correct / total).item()
    class_accuracies = {clsname: class_correct[clsname]/class_total[clsname]
                        for clsname in classes}

    return accuracy, class_accuracies

In [28]:
def inference(model, data):
    outputs = model(input_ids=data["input_ids"], attention_mask=data["attention_mask"])

In [29]:
myModel = EmotionBiLSTM(vocab_size, 250, 64, 13)

In [30]:
train_model(myModel, dataloaderTrain, 100)

Epoch  1, iteration 1376:  Loss = 1.876  Iteration time = 426.411
Epoch  2, iteration 2751:  Loss = 1.760  Iteration time = 419.290
Epoch  3, iteration 4126:  Loss = 1.660  Iteration time = 420.622
Epoch  4, iteration 5501:  Loss = 1.277  Iteration time = 420.639
Epoch  5, iteration 6876:  Loss = 1.616  Iteration time = 421.890
Epoch  6, iteration 8251:  Loss = 1.340  Iteration time = 421.678
Epoch  7, iteration 9626:  Loss = 1.139  Iteration time = 422.531
Epoch  8, iteration 11001:  Loss = 1.272  Iteration time = 421.122
Epoch  9, iteration 12376:  Loss = 1.049  Iteration time = 424.323
Epoch 10, iteration 13751:  Loss = 0.966  Iteration time = 423.213
Epoch 11, iteration 15126:  Loss = 0.803  Iteration time = 423.866
Epoch 12, iteration 16501:  Loss = 1.603  Iteration time = 419.989
Epoch 13, iteration 17876:  Loss = 1.157  Iteration time = 423.799
Epoch 14, iteration 19251:  Loss = 1.249  Iteration time = 421.980
Epoch 15, iteration 20626:  Loss = 1.153  Iteration time = 421.063
Ep

In [31]:
dataSetTest = EmotionTextDataset(X_test, y_train)
dataloaderTest = tud.DataLoader(dataSetTest, batch_size=64, shuffle=True)

In [32]:
test_model(myModel, dataloaderTest)

TypeError: 'int' object is not iterable

In [33]:
# Save model weights
torch.save(myModel.state_dict(), "emotion_model.pt")

In [34]:
#Save the architecture and weights
torch.save(myModel, "emotion_model_full.pt")

In [37]:
drive.mount('/content/drive')

# Save to your Drive
torch.save(myModel.state_dict(), '/content/drive/MyDrive/emotion_model.pt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
# Save to your Drive
torch.save(myModel, '/content/drive/MyDrive/emotion_model_full.pt')